## GPT API calls for Direction 2 Constraint generation

In [5]:
system_prompt = """You are an English writing expert and you need to set hard essay prompts for the genre of Relistic Fiction. Your aim is to generate extremely hard constraints that make the essay writing very challenging. 

An essay prompt is defined as a main Instruction + Constraints. You will be given an Instruction as an input and you should generate more complex constraints that can be added to the Instruction. 

Constraints can be defined in the following categories - 
Content - Egs: Climate change impacts, .
Linguistic - Egs: Victorian English, Pirate slang.
Style - Egs: Respectful Tone, ELI5
Format - Egs: List 3 bullet points, paragraphs.
Number - Egs: 500 word essay, three arguments for electric vehicles.

The definitions are just for your reference. You do not 

Give me a numbered list of EXACTLY {} CONSTRAINTS.

You may include all or even just one category based on what's appropriate for the Instruction. You do not need to make sure you include all categories. The definitions are just for your reference. 

The constraints should be complex and wildly creative so you may come up with constraints that can differ from the main topic of the Instruction. But keep them realistic enough and not too contradictory to make it possible to write a story. It should NOT require any specific domain knowledge to understand the constraints. DO NOT INCLUDE COMPLICATED VOCABULARY OR POETRY RELATED TERMINOLOGY in your responses. Make the constraints such that a flow (if any) in a story can be maintained.

The constraints should be clear and atomic, that is, if a constraint can be decomposed into multiple sub constraints, list all of them separately. DO NOT REPEAT CONSTRAINTS. DO NOT START YOUR LINE WITH THE CONSTRAINT CATEGORY (Lisguistic:, Format:, etc), JUST GIVE ONLY THE CONSTRAINT.
"""

prompt_examples = """Here are some examples -

Input - Here are some examples -

Input - Write a story that follows the journey of a professional woman of color named Rachel Michelle.
Output - 
1. Rachel is single, has two kids she is supporting through high school.
2. Rachel is considering leaving her corporate job to become an entrepreneur. 
3. The story should detail the steps she takes to create a life plan and make the leap.
4. Highlight her success as an entrepreneur with five employees.
5. The book should be written in a motivational and engaging style.
6. Incorporate practical and mental wellness strategies that helped her achieve her goals.
7. Explore the complex relationship between her and her growing kids.
8. The story should be a lesson in holistic wellness and life coaching.
9. The story should cater to a young audience.
10. Write the story in 8 paragraphs or less.

Input - Write a story about the incident of two friend's day out gone wrong.
Output - 
1. One of the friends must have a secret that is revealed during the day out.
2. One of the friends must have a phobia that becomes a central theme in the story.
3. Maintain a casual tone overall but create tension when required to keep readers engaged. 
4. Involve their pet parrot in the plot. 
5. The story must contain an unexpected plot twist.
6. The friends must communicate through handwritten notes for a part of the story.
7. The story must include a mysterious stranger who impacts their day.
8. The setting of the story must change at least two times.
9. The climax of the story must involve a natural disaster.
10. The story must end with a cliffhanger that leaves the readers guessing about the fate of the friends.

"""
user_input = """
Input - {}
Output -
"""

system_prompt = system_prompt + prompt_examples

Setup OpenAI Model

In [6]:
from openai import OpenAI
from datetime import datetime
import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.environ['OPENAI_API_KEY']
client = OpenAI(api_key=api_key)

def chat(instruction, model="gpt-3.5-turbo", system_prompt=system_prompt, n=20, log=False):
    
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content":system_prompt.format(n)},
            {"role": "user", "content":user_input.format(instruction)},
        ]
    )

    log_usage(tokens=response.usage.total_tokens, model=model)

    if log:
        print("Total tokens used: ", response.usage.total_tokens)

    return response


def log_usage(tokens, model):
    # Get the current date and time
    current_time = datetime.now().strftime("%m-%d-%Y %H:%M:%S")

    # Write the date-time and tokens used to the file
    with open("api_usage.txt", "a") as file:
        file.write(f"{model} {current_time} : {tokens}\n")

def total_usage():
    model_tokens = {}
    model_prices = {"gpt-3.5-turbo": 0.0015, "gpt-4-turbo": 0.03}
    
    with open("api_usage.txt", "r") as file:
        for line in file:
            parts = line.split(" ")
            if len(parts) >= 4:
                model_name = parts[0]
                tokens_str = parts[-1].strip()
                tokens = int(tokens_str)
                if model_name in model_tokens:
                    model_tokens[model_name] += tokens
                else:
                    model_tokens[model_name] = tokens
    
    total_cost = 0
    for model, tokens in model_tokens.items():
        cost = (tokens * model_prices[model]) / 1000
        total_cost += cost
        print(f"Total tokens used for {model}: {tokens}")
        print(f"\nTotal cost for {model}: {cost}$")
    
    print(f"\nTotal cost so far: {total_cost}$")
    return


In [7]:
import openpyxl
import pandas as pd

def get_instructions():

    # Read the Excel file into a DataFrame
    df = pd.read_excel("Data/input_instructions.xlsx")

    instructions = [list(row.values()) for row in df.to_dict(orient='records')]
    
    return instructions

In [9]:
def generate_constraints(sheet_name):
    n = 20
    instructions = get_instructions()
    sheet_name = "chatgpt-3.5-turbo"

    df = pd.read_excel('Data/constraints.xlsx', sheet_name=sheet_name)

    dfs_to_concat = []

    for instruction in instructions:
        
        response = chat(instruction=instruction[0], n=n, log=True)

        new_row = {
            'Instruction': instruction[0],
            'Category': instruction[1],
            'Constraints': response.choices[0].message.content
        }
        new_df = pd.DataFrame([new_row])

        dfs_to_concat.append(new_df)
    
    # Concatenate all DataFrames in the list
    df = pd.concat([df] + dfs_to_concat, ignore_index=True)

    df.to_excel('Data/constraints.xlsx', index=False, sheet_name=sheet_name)

    

In [11]:
generate_constraints()

Total tokens used:  1275
Total tokens used:  1134
Total tokens used:  1170
Total tokens used:  1223
Total tokens used:  1158
Total tokens used:  1160
Total tokens used:  1168
Total tokens used:  1162
Total tokens used:  1204
Total tokens used:  1168
Total tokens used:  1164
Total tokens used:  1242


In [12]:
total_usage()

Total tokens used for gpt-3.5-turbo: 43194

Total cost for gpt-3.5-turbo: 0.064791$

Total cost so far: 0.064791$


In [153]:
instructions = get_instructions()
n = 20
for instruction in instructions:
    # print(instruction[0])
    messages=[
        {"role": "user", "content":user_input.format(instruction[0])},
    ]
    print(messages)

[{'role': 'user', 'content': '\nInput - Plan for a three day road trip for May long weekend from Westfield, New Jersey\nOutput -\n'}]
[{'role': 'user', 'content': '\nInput - Write a story about incident of two friend’s day out going wrong.\nOutput -\n'}]
[{'role': 'user', 'content': '\nInput - Write a short paragraph about why you should not have both a pet cat and a pet bird.\t\nOutput -\n'}]
[{'role': 'user', 'content': '\nInput - Write a passage for ideas for the most amazing welcome party in high school.\nOutput -\n'}]
[{'role': 'user', 'content': '\nInput - Plan a weekend long vacation to Madrid, and mention multiple specific places you would visit and how you would travel in the city\t\nOutput -\n'}]
[{'role': 'user', 'content': '\nInput - Write a story about being stuck in a locked gym storage room with the person you arguably hate the most.\nOutput -\n'}]
[{'role': 'user', 'content': '\nInput - Write a story about returning something a random stranger dropped on the train, late